# Obtain weekday prototypes
The main objective of this task is to create every weekday prototypes. We want to model two types of days based on the consumption activity of each building:
- **Active** day.
- **Inactive** day.

Thus, for each counter on the database, we'll get 13 day prototypes (6 working days * 2 types of days + 1 inactive day corresponding to Sundays). Moreover, there are 97 different buildings, so we expect to get 13 * 97 prototypical days.

This previouisly mentioned activity will be defined by calculating the mean of Sundays' consumptions for each building (greater than this value plus some margin will indicate an active day; lower or equal than this value plus the margin should be labelled as an inactive day).

#### Directory structure
./<br></br>
notebook/<br></br>
    &emsp;|--- data-preprocessing<br></br>
    &emsp;&emsp;&emsp;&emsp;|--- weekday_prototypes.ipynb<br></br>
out/<br></br>
    &emsp;|--- raw_consumptions.zip

In [1]:
CONS_PATH = 'C:/Users/thmas/OneDrive - Universidad de Castilla-La Mancha/Informática/TFG/out/'

In [2]:
import pandas as pd
import numpy as np

In [3]:
counter_id = 27 # Counter ID example

raw = pd.read_pickle(CONS_PATH + 'raw_consumptions.zip')
raw

,building_id,weekday,consumptions
day,,,
2012-02-24,89,4,"[nan, nan, nan, nan, 0.0, 25.9682072759303, 34..."
2012-02-25,89,5,"[8.0, 8.56965980289508, 7.83041664589254, 7.83..."
2012-02-26,89,6,"[9.0, 9.0, 8.47872481882854, 8.52127518117146,..."
2012-02-27,89,0,"[9.93594069444675, 9.0, 10.0, 18.4133936140153..."
2012-02-28,89,1,"[15.0, 15.0, 15.0, 23.0, 41.3474893206788, 39...."
...,...,...,...
2020-03-23,2233,0,"[9.23106212240757, 15.7486593144312, 19.269211..."
2020-03-24,2233,1,"[9.5668751135038, 15.3377513735436, 20.2699453..."
2020-03-25,2233,2,"[9.22960071120226, 15.9943441867809, 20.594844..."


In [4]:
raw_df = raw[raw['building_id'] == counter_id]
raw_df

,building_id,weekday,consumptions
day,,,
2011-07-26,27,1,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2011-07-27,27,2,"[17.0, 19.0, 18.3507946535444, 35.846312818818..."
2011-07-28,27,3,"[18.8887041808661, 18.8030088936913, 18.845892..."
2011-07-29,27,4,"[20.0, 21.0, 20.0, 37.7887789876153, 45.845704..."
2011-07-30,27,5,"[17.2981132075472, 17.0, 17.2396974482587, 17...."
...,...,...,...
2020-03-23,27,0,"[10.1170330737468, 10.9676878955827, 10.967739..."
2020-03-24,27,1,"[10.8590892649269, 10.9677121385118, 10.967712..."
2020-03-25,27,2,"[10.9677235262438, 10.3559814000229, 10.579513..."


### Obtaining prototype measures
In order to get the required measures for every day, first we obtain them for sundays, which are supposed to be inactive days. After that, we base the active/inactive result depending on:

- **Inactive days** &rarr; daily consumption mean within [sundays.mean + 2 * sundays.std, +$\infty$)
- **Active days** &rarr; daily consumption mean within [0, sundays.mean + 2 * sundays.std)

We'll store all this prototypical days (every building has 13, as previously discussed) in a pandas DataFrame for later use

In [5]:
def dropNan(df: pd.DataFrame) -> pd.DataFrame:
    nan_rows = []
    
    for i in range(df.shape[0]):
        consumptions = df['consumptions'].iloc[i]
        
        if True in np.isnan(consumptions):
            nan_rows.append(df.index[i])
            
    return df.drop(index=nan_rows)

#     return df[df['consumptions'].map(lambda consumption: np.nan not in consumption)]

dropNan(raw_df)

,building_id,weekday,consumptions
day,,,
2011-07-27,27,2,"[17.0, 19.0, 18.3507946535444, 35.846312818818..."
2011-07-28,27,3,"[18.8887041808661, 18.8030088936913, 18.845892..."
2011-07-29,27,4,"[20.0, 21.0, 20.0, 37.7887789876153, 45.845704..."
2011-07-30,27,5,"[17.2981132075472, 17.0, 17.2396974482587, 17...."
2011-07-31,27,6,"[18.8455988695869, 17.1538525475854, 18.0, 18...."
...,...,...,...
2020-03-23,27,0,"[10.1170330737468, 10.9676878955827, 10.967739..."
2020-03-24,27,1,"[10.8590892649269, 10.9677121385118, 10.967712..."
2020-03-25,27,2,"[10.9677235262438, 10.3559814000229, 10.579513..."


In [6]:
def get_threshold(df: pd.DataFrame) -> float:
    df['daily'] = df['consumptions'].apply(np.sum)
    
    return df['daily'].mean()

In [7]:
def get_prototype(df: pd.DataFrame, counter_id: int, weekday: int, active: bool, type: str = 'mean') -> pd.DataFrame:
    cons = []
    for i in range(24):
        i_consumptions = []
        for j in range(df.shape[0]):
            i_consumptions.append(df['consumptions'].iloc[j][i])

        if type == 'std':
            cons.append(np.std(i_consumptions))
        else:
            cons.append(np.mean(i_consumptions))
            
    return pd.DataFrame({'building_id': counter_id, 'weekday': weekday, 'active': active, 'consumptions': [cons]})

In [8]:
clean_df = dropNan(raw_df)

sundays = clean_df[clean_df['weekday'] == 6] # Select Sundays
sundays

,building_id,weekday,consumptions
day,,,
2011-07-31,27,6,"[18.8455988695869, 17.1538525475854, 18.0, 18...."
2011-08-07,27,6,"[11.9497448173823, 11.0502551826177, 11.641160..."
2011-08-14,27,6,"[11.8460809575264, 11.9595877316931, 12.0, 12...."
2011-08-21,27,6,"[11.0, 11.9707754966337, 11.0292245033663, 11...."
2011-08-28,27,6,"[11.0, 11.0, 11.0, 11.0, 11.0, 12.0, 12.0, 13...."
...,...,...,...
2020-02-23,27,6,"[21.9461547109486, 21.5688397657194, 21.323437..."
2020-03-01,27,6,"[20.9462068881718, 21.9461609012988, 20.946213..."
2020-03-08,27,6,"[19.9461339838782, 18.9462003523115, 19.946239..."


In [9]:
threshold = get_threshold(sundays)

mean_proto = get_prototype(sundays, counter_id, 6, False, type='mean') # Get Sundays prototype
mean_proto

D:\PythonEnvironments\pytfg\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,building_id,weekday,active,consumptions
0,27,6,False,"[21.745047367917923, 21.641706434645418, 21.64..."


In [10]:
std_proto = get_prototype(sundays, counter_id, 6, False, type='std') # Get Sundays prototype
std_proto

,building_id,weekday,active,consumptions
0,27,6,False,"[6.041029206926876, 6.067249149649222, 6.04414..."


In [11]:
for i in range(0, 6):
    df = clean_df[clean_df['weekday'] == i]
    df['daily'] = df['consumptions'].apply(np.sum)
    
    df_a = df.loc[df['daily'] > threshold] # Select active days
    mean_proto = mean_proto.append(get_prototype(df_a, counter_id, i, True, type='mean'))
    std_proto = std_proto.append(get_prototype(df_a, counter_id, i, True, type='std'))
    
    df_i = df.loc[df['daily'] <= threshold] # Select inactive days
    mean_proto = mean_proto.append(get_prototype(df_i, counter_id, i, False, type='mean'))
    std_proto = std_proto.append(get_prototype(df_i, counter_id, i, False, type='std'))

mean_proto.reset_index(drop=True, inplace=True), std_proto.reset_index(drop=True, inplace=True)
mean_proto

D:\PythonEnvironments\pytfg\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,building_id,weekday,active,consumptions
0,27,6,False,"[21.745047367917923, 21.641706434645418, 21.64..."
1,27,0,True,"[22.730189629410912, 24.089560330388434, 31.08..."
2,27,0,False,"[16.679401791104826, 16.563127736496302, 17.23..."
3,27,1,True,"[24.30827263579404, 25.598547213870944, 32.549..."
4,27,1,False,"[16.053105096121442, 16.22752645506319, 16.683..."
5,27,2,True,"[24.710433864649467, 25.970457092593225, 33.09..."
6,27,2,False,"[16.09883069727714, 15.91585086885025, 16.5417..."
7,27,3,True,"[24.66647902605235, 25.868352809869233, 33.105..."
8,27,3,False,"[16.346460832160904, 16.311245857993576, 16.92..."
9,27,4,True,"[24.504470552802474, 25.614175760062118, 32.76..."
